In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('trips17small.csv')
df

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,new_TaxiID
0,05/12/2017 09:00:00 PM,05/12/2017 09:15:00 PM,865.0,4.6,1.703132e+10,1.703107e+10,32.0,7.0,15.00,2.55,...,19.55,Credit Card,American United,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.929047,-87.651311,POINT (-87.6513108767 41.9290469366),2464
1,05/13/2017 10:00:00 AM,05/13/2017 10:30:00 AM,1588.0,13.5,NaN,NaN,NaN,NaN,35.25,8.81,...,44.06,Credit Card,American United,NaN,NaN,NaN,NaN,NaN,NaN,3143
2,05/13/2017 01:45:00 PM,05/13/2017 02:15:00 PM,2184.0,12.7,NaN,NaN,NaN,NaN,35.50,0.00,...,40.50,Cash,American United,NaN,NaN,NaN,NaN,NaN,NaN,3143
3,05/13/2017 02:30:00 PM,05/13/2017 02:30:00 PM,214.0,0.4,NaN,NaN,NaN,NaN,4.75,0.00,...,4.75,Cash,American United,NaN,NaN,NaN,NaN,NaN,NaN,3143
4,05/13/2017 04:30:00 PM,05/13/2017 04:45:00 PM,1217.0,7.8,NaN,NaN,NaN,NaN,22.75,4.75,...,28.50,Credit Card,American United,NaN,NaN,NaN,NaN,NaN,NaN,3143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6195692,10/08/2017 04:30:00 PM,10/08/2017 05:45:00 PM,4260.0,28.5,1.703184e+10,1.703198e+10,32.0,56.0,72.25,0.00,...,72.25,Dispute,Choice Taxi Association,41.871016,-87.631407,POINT (-87.6314065252 41.8710158803),41.785999,-87.750934,POINT (-87.7509342894 41.785998518),758
6195693,10/22/2017 06:00:00 PM,10/22/2017 06:15:00 PM,540.0,0.0,1.703132e+10,1.703183e+10,32.0,28.0,7.25,1.65,...,9.90,Credit Card,Blue Ribbon Taxi Association Inc.,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.885281,-87.657233,POINT (-87.6572331997 41.8852813201),4246
6195694,10/23/2017 11:15:00 AM,10/23/2017 11:15:00 AM,360.0,0.0,NaN,NaN,7.0,6.0,6.00,3.00,...,9.00,Credit Card,Blue Ribbon Taxi Association Inc.,41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),41.944227,-87.655998,POINT (-87.6559981815 41.9442266014),2906
6195695,10/07/2017 01:45:00 PM,10/07/2017 02:00:00 PM,900.0,3.1,1.703108e+10,1.703108e+10,8.0,8.0,12.25,2.65,...,15.90,Credit Card,Star North Management LLC,41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),4986


# Achtung folgende zelle dauert etwa 10 min

In [3]:
#from am/pm to 24 hour time
df.loc[:, 'Trip Start Timestamp']=pd.to_datetime(df['Trip Start Timestamp'])
#from am/pm to 24 hour time
df.loc[:, 'Trip End Timestamp']=pd.to_datetime(df['Trip End Timestamp'])

In [4]:
#dropt alle zeilen, deren end timestamp vor der start timestamp sind, dies sind nur 30 eintraege also
#statistisch nicht relevant und auch kein durchlaufender gleicher fehler zu erkennen, mit dem man die daten
#"reparieren koennte"
df=df.drop(df[df['Trip End Timestamp']<df['Trip Start Timestamp']].index)

In [5]:
#kreirt zwei dummy variablen, zum vergleichen spaeter
tage=["2017-01-01 00:00:00",'2017-01-02 00:00:00',"2017-01-01 00:00:00",'2017-01-01 16:00:00']
tage=pd.to_datetime(tage)
tag=tage[1]-tage[0]
halbertag=tage[3]-tage[2]

In [6]:
halbertag

Timedelta('0 days 16:00:00')

In [7]:
df[df['Trip End Timestamp']-df['Trip Start Timestamp']>halbertag]

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,new_TaxiID
74567,2017-06-07 12:30:00,2017-06-08 07:30:00,68665.0,19.80,NaN,NaN,NaN,NaN,0.01,0.00,...,0.01,Cash,303 Taxi,NaN,NaN,NaN,NaN,NaN,NaN,3186
606291,2017-01-11 07:30:00,2017-01-12 00:00:00,59580.0,0.70,NaN,NaN,NaN,NaN,5.00,0.00,...,5.00,Cash,Sun Taxi,NaN,NaN,NaN,NaN,NaN,NaN,6888
607179,2017-01-12 07:15:00,2017-01-13 00:00:00,59940.0,1.50,NaN,NaN,NaN,NaN,7.50,0.00,...,7.50,Cash,Sun Taxi,NaN,NaN,NaN,NaN,NaN,NaN,1336
608720,2017-01-13 15:30:00,2017-01-14 09:30:00,64629.0,0.00,NaN,NaN,NaN,NaN,18.25,0.00,...,18.25,Cash,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,876
617051,2017-01-28 03:15:00,2017-01-29 00:00:00,74580.0,1.50,NaN,NaN,NaN,NaN,6.75,0.00,...,6.75,Cash,Globe Taxi,NaN,NaN,NaN,NaN,NaN,NaN,4436
634428,2017-02-11 05:45:00,2017-02-12 00:15:00,67107.0,0.00,NaN,NaN,NaN,NaN,61.00,0.00,...,61.00,Cash,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,3035
639502,2017-02-13 10:15:00,2017-02-14 10:15:00,85672.0,0.00,NaN,NaN,60.0,NaN,3.25,0.00,...,3.25,Cash,Chicago Carriage Cab Corp,41.836150,-87.648788,POINT (-87.6487879519 41.8361501547),NaN,NaN,NaN,6261
644042,2017-02-23 11:15:00,2017-02-24 09:45:00,81319.0,0.00,NaN,NaN,NaN,NaN,3.25,0.00,...,3.25,Cash,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,1841
654170,2017-05-03 12:45:00,2017-05-04 11:30:00,82177.0,0.00,NaN,NaN,41.0,NaN,3.25,0.00,...,3.25,Cash,Chicago Carriage Cab Corp,41.794090,-87.592311,POINT (-87.592310855 41.794090253),NaN,NaN,NaN,6341
670754,2017-05-22 07:15:00,2017-05-23 00:00:00,60000.0,0.50,NaN,NaN,NaN,NaN,4.50,2.00,...,6.50,Cash,City Service,NaN,NaN,NaN,NaN,NaN,NaN,3617


In [8]:
#
df=df.drop(df[df['Trip End Timestamp']-df['Trip Start Timestamp']>halbertag].index)

In [9]:
df.count()

Trip Start Timestamp          6195628
Trip End Timestamp            6195000
Trip Seconds                  6194993
Trip Miles                    6195612
Pickup Census Tract           4170210
Dropoff Census Tract          4144938
Pickup Community Area         5447801
Dropoff Community Area        5335168
Fare                          6195603
Tips                          6195603
Tolls                         5953061
Extras                        6195603
Trip Total                    6195603
Payment Type                  6195628
Company                       6195628
Pickup Centroid Latitude      5448087
Pickup Centroid Longitude     5448087
Pickup Centroid Location      5448087
Dropoff Centroid Latitude     5351386
Dropoff Centroid Longitude    5351386
Dropoff Centroid  Location    5351386
new_TaxiID                    6195628
dtype: int64

In [10]:
df=df.drop_duplicates()

In [11]:
df.count()

Trip Start Timestamp          6186046
Trip End Timestamp            6185421
Trip Seconds                  6185414
Trip Miles                    6186030
Pickup Census Tract           4166098
Dropoff Census Tract          4140879
Pickup Community Area         5443905
Dropoff Community Area        5331324
Fare                          6186021
Tips                          6186021
Tolls                         5947211
Extras                        6186021
Trip Total                    6186021
Payment Type                  6186046
Company                       6186046
Pickup Centroid Latitude      5444190
Pickup Centroid Longitude     5444190
Pickup Centroid Location      5444190
Dropoff Centroid Latitude     5347541
Dropoff Centroid Longitude    5347541
Dropoff Centroid  Location    5347541
new_TaxiID                    6186046
dtype: int64

In [12]:
df=df.drop(df[df['Trip End Timestamp'].isna()].index)

In [13]:
df.count()

Trip Start Timestamp          6185421
Trip End Timestamp            6185421
Trip Seconds                  6185414
Trip Miles                    6185405
Pickup Census Tract           4165632
Dropoff Census Tract          4140879
Pickup Community Area         5443335
Dropoff Community Area        5331324
Fare                          6185396
Tips                          6185396
Tolls                         5946586
Extras                        6185396
Trip Total                    6185396
Payment Type                  6185421
Company                       6185421
Pickup Centroid Latitude      5443620
Pickup Centroid Longitude     5443620
Pickup Centroid Location      5443620
Dropoff Centroid Latitude     5347541
Dropoff Centroid Longitude    5347541
Dropoff Centroid  Location    5347541
new_TaxiID                    6185421
dtype: int64

In [113]:
#unter 20 eintraege werden geloescht, und sie sind alle eher klein/oder sogar fehlerhaft
df=df.drop(df[df['Trip Seconds'].isna()].index)
df['dur_min']=df['Trip Seconds'].apply(lambda x:round(x/60,2))

In [114]:
df.count()

Trip Start Timestamp          5916168
Trip End Timestamp            5916168
Trip Seconds                  5916168
Trip Miles                    5916164
Pickup Census Tract           4066643
Dropoff Census Tract          4055435
Pickup Community Area         5297080
Dropoff Community Area        5202505
Fare                          5916168
Tips                          5916168
Tolls                         5916168
Extras                        5916168
Trip Total                    5916168
Payment Type                  5916168
Company                       5916168
Pickup Centroid Latitude      5297278
Pickup Centroid Longitude     5297278
Pickup Centroid Location      5297278
Dropoff Centroid Latitude     5218640
Dropoff Centroid Longitude    5218640
Dropoff Centroid  Location    5218640
new_TaxiID                    5916168
dur_min                       5916168
dtype: int64

In [16]:
df=df.drop(df[df['Fare'].isna()].index)

In [17]:
df.count()

Trip Start Timestamp          6185389
Trip End Timestamp            6185389
Trip Seconds                  6185389
Trip Miles                    6185378
Pickup Census Tract           4165621
Dropoff Census Tract          4140867
Pickup Community Area         5443321
Dropoff Community Area        5331309
Fare                          6185389
Tips                          6185389
Tolls                         5946584
Extras                        6185389
Trip Total                    6185389
Payment Type                  6185389
Company                       6185389
Pickup Centroid Latitude      5443606
Pickup Centroid Longitude     5443606
Pickup Centroid Location      5443606
Dropoff Centroid Latitude     5347526
Dropoff Centroid Longitude    5347526
Dropoff Centroid  Location    5347526
new_TaxiID                    6185389
dtype: int64

In [18]:
#schon beobachtet, dass von dieses 269.070 trips alle die gleichen start und end timestamp haben
#weitere beobachtung= nur 2198 davon trip miles>1, 2997 haben trip mile >0.5 , 3842 haben trip mile>0.2
notime=df[df['Trip Seconds']==0]

In [19]:
notime

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,new_TaxiID
1065,2017-05-09 08:45:00,2017-05-09 08:45:00,0.0,0.1,NaN,NaN,NaN,NaN,3.50,0.0,...,3.50,Cash,Flash Cab,NaN,NaN,NaN,NaN,NaN,NaN,2941
1426,2017-05-07 21:15:00,2017-05-07 21:15:00,0.0,0.0,NaN,NaN,NaN,NaN,3.25,0.0,...,3.25,Cash,Flash Cab,NaN,NaN,NaN,NaN,NaN,NaN,5448
1468,2017-05-15 19:00:00,2017-05-15 19:00:00,0.0,0.8,NaN,NaN,NaN,NaN,5.75,0.0,...,5.75,Cash,Flash Cab,NaN,NaN,NaN,NaN,NaN,NaN,1949
1509,2017-05-13 16:30:00,2017-05-13 16:30:00,0.0,0.0,NaN,NaN,NaN,NaN,3.25,0.0,...,3.25,Cash,Flash Cab,NaN,NaN,NaN,NaN,NaN,NaN,2405
1768,2017-05-21 19:30:00,2017-05-21 19:30:00,0.0,0.0,NaN,NaN,NaN,NaN,3.25,0.0,...,3.25,Cash,Flash Cab,NaN,NaN,NaN,NaN,NaN,NaN,1949
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6195455,2017-10-23 03:15:00,2017-10-23 03:15:00,0.0,0.0,NaN,NaN,8.0,8.0,3.50,11.0,...,64.50,Credit Card,Chicago Medallion Management,41.899602,-87.633308,POINT (-87.6333080367 41.899602111),41.899602,-87.633308,POINT (-87.6333080367 41.899602111),672
6195493,2017-10-23 20:30:00,2017-10-23 20:30:00,0.0,11.1,1.703198e+10,1.703198e+10,76.0,76.0,3.25,0.0,...,3.25,Cash,Star North Management LLC,41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),41.979071,-87.903040,POINT (-87.9030396611 41.9790708201),6433
6195528,2017-10-23 09:00:00,2017-10-23 09:00:00,0.0,0.0,NaN,NaN,39.0,39.0,3.25,0.0,...,3.25,Cash,Taxi Affiliation Services,41.808916,-87.596183,POINT (-87.5961833442 41.8089162826),41.808916,-87.596183,POINT (-87.5961833442 41.8089162826),2071
6195594,2017-10-22 17:45:00,2017-10-22 17:45:00,0.0,0.0,1.703184e+10,1.703184e+10,32.0,32.0,3.25,0.0,...,3.75,Cash,Blue Ribbon Taxi Association Inc.,41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),41.880994,-87.632746,POINT (-87.6327464887 41.8809944707),4246


In [20]:
notime[notime['Trip Miles'].isna()]


,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,new_TaxiID
235587,2017-01-08 18:30:00,2017-01-08 18:30:00,0.0,NaN,NaN,NaN,NaN,NaN,3.25,0.00,...,3.75,Cash,Top Cab Affiliation,NaN,NaN,NaN,NaN,NaN,NaN,1604
1569616,2017-02-14 14:45:00,2017-02-14 14:45:00,0.0,NaN,1.703116e+10,1.703116e+10,16.0,16.0,3.25,0.00,...,3.75,Cash,Northwest Management LLC,41.958798,-87.738394,POINT (-87.7383939059 41.9587978628),41.958798,-87.738394,POINT (-87.7383939059 41.9587978628),844
3042180,2017-05-11 19:45:00,2017-05-11 19:45:00,0.0,NaN,NaN,NaN,16.0,16.0,3.25,0.00,...,3.25,Cash,Northwest Management LLC,41.953582,-87.723452,POINT (-87.7234523905 41.9535821253),41.953582,-87.723452,POINT (-87.7234523905 41.9535821253),6527
3503492,2017-05-22 13:45:00,2017-05-22 13:45:00,0.0,NaN,NaN,NaN,16.0,16.0,3.25,0.00,...,3.75,Cash,Dispatch Taxi Affiliation,41.953582,-87.723452,POINT (-87.7234523905 41.9535821253),41.953582,-87.723452,POINT (-87.7234523905 41.9535821253),2148
4569393,2017-07-07 16:00:00,2017-07-07 16:00:00,0.0,NaN,1.703116e+10,1.703116e+10,16.0,16.0,3.25,0.00,...,4.25,Cash,Dispatch Taxi Affiliation,41.958798,-87.738394,POINT (-87.7383939059 41.9587978628),41.958798,-87.738394,POINT (-87.7383939059 41.9587978628),5402
5267290,2017-08-23 17:15:00,2017-08-23 17:15:00,0.0,NaN,NaN,NaN,21.0,21.0,3.25,0.00,...,3.25,Cash,4053 - 40193 Adwar H. Nikola,41.938666,-87.711211,POINT (-87.7112105933 41.93866619620001),41.938666,-87.711211,POINT (-87.7112105933 41.93866619620001),1072
6127994,2017-10-20 15:45:00,2017-10-20 15:45:00,0.0,NaN,NaN,NaN,21.0,21.0,3.25,0.65,...,4.90,Unknown,1469 - 64126 Omar Jada,41.938666,-87.711211,POINT (-87.7112105933 41.93866619620001),41.938666,-87.711211,POINT (-87.7112105933 41.93866619620001),5143


In [21]:
notime[notime['Trip Miles']<0.2].index

Int64Index([   1065,    1426,    1509,    1768,    1833,    2414,    3039,
               3059,    3456,    3459,
            ...
            6195208, 6195213, 6195214, 6195240, 6195282, 6195295, 6195455,
            6195528, 6195594, 6195598],
           dtype='int64', length=264159)

In [22]:
#schmeiße alles raus was keine trip seconds hat, denn die 4 tausend werte, wo nicht noch die trip miles
#auch fast null sind nicht sehr relevant sind
df=df.drop(df[df['Trip Seconds']==0].index)

In [23]:
df.count()

Trip Start Timestamp          5916326
Trip End Timestamp            5916326
Trip Seconds                  5916326
Trip Miles                    5916322
Pickup Census Tract           4066730
Dropoff Census Tract          4055522
Pickup Community Area         5297211
Dropoff Community Area        5202634
Fare                          5916326
Tips                          5916326
Tolls                         5680416
Extras                        5916326
Trip Total                    5916326
Payment Type                  5916326
Company                       5916326
Pickup Centroid Latitude      5297409
Pickup Centroid Longitude     5297409
Pickup Centroid Location      5297409
Dropoff Centroid Latitude     5218769
Dropoff Centroid Longitude    5218769
Dropoff Centroid  Location    5218769
new_TaxiID                    5916326
dtype: int64

In [87]:
test['sum']=df[['Tips','Extras','Tolls','Fare']].sum(axis=1)
test[test['sum']-test['Trip Total']>0.001]

<ipython-input-87-5fc624b7d359>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['sum']=df[['Tips','Extras','Tolls','Fare']].sum(axis=1)


,Tips,Extras,Tolls,Fare,Trip Total,sum


In [76]:
#since fare+tipps+tolls+extras=Trip Total bei allen stimmt(und nan=0)kann 
#ich die fehlenden tolls mit nullen auffuellen
df.loc[:,'Tolls']=df.loc[:,'Tolls'].fillna(0)

In [77]:
df.count()

Trip Start Timestamp          5916326
Trip End Timestamp            5916326
Trip Seconds                  5916326
Trip Miles                    5916322
Pickup Census Tract           4066730
Dropoff Census Tract          4055522
Pickup Community Area         5297211
Dropoff Community Area        5202634
Fare                          5916326
Tips                          5916326
Tolls                         5916326
Extras                        5916326
Trip Total                    5916326
Payment Type                  5916326
Company                       5916326
Pickup Centroid Latitude      5297409
Pickup Centroid Longitude     5297409
Pickup Centroid Location      5297409
Dropoff Centroid Latitude     5218769
Dropoff Centroid Longitude    5218769
Dropoff Centroid  Location    5218769
new_TaxiID                    5916326
dtype: int64

In [103]:
#hier wird geldwaesche betrieben, ueber 1000 euro zahlen und unter 3 stunden unterwegs zu sein macht keinen sinn.
#es sind nur 158 fahrten also wahrscheinich nicht sehr aussage kraeftig, aber solche ausreiser aus dem datensatz
#brauchen wir trotzdem nicht
test=df[df['Trip Total']>1000]
delete=test[test['Trip Seconds']<18000].index
df=df.drop(delete)

In [104]:
df.count()

Trip Start Timestamp          5916168
Trip End Timestamp            5916168
Trip Seconds                  5916168
Trip Miles                    5916164
Pickup Census Tract           4066643
Dropoff Census Tract          4055435
Pickup Community Area         5297080
Dropoff Community Area        5202505
Fare                          5916168
Tips                          5916168
Tolls                         5916168
Extras                        5916168
Trip Total                    5916168
Payment Type                  5916168
Company                       5916168
Pickup Centroid Latitude      5297278
Pickup Centroid Longitude     5297278
Pickup Centroid Location      5297278
Dropoff Centroid Latitude     5218640
Dropoff Centroid Longitude    5218640
Dropoff Centroid  Location    5218640
new_TaxiID                    5916168
dtype: int64

## ab hier koennte man nochmal besprechen, ob man etwas mit diesen nicht angefangenen fahrten macht? sind halt doch einige die man noch löschen könnte

In [24]:
df[df['Fare']<3.25]

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,new_TaxiID
2925,2017-04-01 09:00:00,2017-04-01 09:00:00,6.0,0.0,NaN,NaN,NaN,NaN,2.00,0.0,...,2.00,Cash,303 Taxi,NaN,NaN,NaN,NaN,NaN,NaN,4403
2926,2017-04-01 09:00:00,2017-04-01 09:00:00,5.0,0.0,NaN,NaN,NaN,NaN,0.01,0.0,...,0.01,Credit Card,303 Taxi,NaN,NaN,NaN,NaN,NaN,NaN,4403
2930,2017-04-01 16:30:00,2017-04-01 16:30:00,7.0,0.0,NaN,NaN,NaN,NaN,0.01,0.0,...,0.01,Credit Card,303 Taxi,NaN,NaN,NaN,NaN,NaN,NaN,4403
2957,2017-04-01 00:45:00,2017-04-01 01:00:00,1480.0,17.7,NaN,NaN,76.0,NaN,0.01,0.0,...,0.01,Cash,303 Taxi,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),NaN,NaN,NaN,2107
3027,2017-04-01 04:30:00,2017-04-01 05:00:00,1604.0,12.6,NaN,NaN,NaN,76.0,0.01,0.0,...,0.01,Cash,303 Taxi,NaN,NaN,NaN,41.980264,-87.913625,POINT (-87.913624596 41.9802643146),980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6179288,2017-10-22 22:30:00,2017-10-22 23:00:00,1560.0,10.9,1.703198e+10,1.703184e+10,56.0,33.0,0.00,0.0,...,0.00,Cash,Taxi Affiliation Services,41.785999,-87.750934,POINT (-87.7509342894 41.785998518),41.849247,-87.624135,POINT (-87.6241352979 41.84924675450001),1380
6180221,2017-10-23 17:15:00,2017-10-23 17:30:00,960.0,0.4,NaN,NaN,7.0,77.0,0.00,0.0,...,0.00,Credit Card,Blue Ribbon Taxi Association Inc.,41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),41.986712,-87.663416,POINT (-87.6634164054 41.9867117999),1247
6180808,2017-10-23 05:45:00,2017-10-23 05:45:00,180.0,0.0,NaN,NaN,7.0,7.0,0.00,0.0,...,0.00,Dispute,Dispatch Taxi Affiliation,41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),1430
6190931,2017-10-23 09:00:00,2017-10-23 09:00:00,60.0,0.0,NaN,NaN,7.0,7.0,0.00,0.0,...,0.00,Credit Card,Choice Taxi Association,41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),3622


In [25]:
nominute=df[df['Trip Seconds']<60]

In [26]:
nominute

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,new_TaxiID
29,2017-04-25 15:45:00,2017-04-25 15:45:00,25.0,0.00,NaN,NaN,NaN,NaN,3.25,0.00,...,31.25,Credit Card,American United,NaN,NaN,NaN,NaN,NaN,NaN,3996
41,2017-04-24 18:00:00,2017-04-24 18:00:00,1.0,0.00,NaN,NaN,NaN,NaN,3.25,0.00,...,3.25,Cash,American United,NaN,NaN,NaN,NaN,NaN,NaN,2512
54,2017-04-27 11:30:00,2017-04-27 11:30:00,20.0,0.00,NaN,NaN,21.0,21.0,3.25,0.65,...,4.90,Credit Card,American United,41.938666,-87.711211,POINT (-87.7112105933 41.93866619620001),41.938666,-87.711211,POINT (-87.7112105933 41.93866619620001),4463
61,2017-04-27 11:00:00,2017-04-27 11:00:00,19.0,0.00,NaN,NaN,NaN,NaN,3.25,0.65,...,4.90,Credit Card,American United,NaN,NaN,NaN,NaN,NaN,NaN,4463
62,2017-04-27 11:15:00,2017-04-27 11:15:00,18.0,0.00,NaN,NaN,NaN,NaN,3.25,0.65,...,4.90,Credit Card,American United,NaN,NaN,NaN,NaN,NaN,NaN,4463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2986910,2017-07-28 20:00:00,2017-07-28 20:00:00,5.0,0.01,NaN,NaN,NaN,NaN,3.25,0.00,...,3.25,Cash,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,4667
2987061,2017-07-28 20:30:00,2017-07-28 20:30:00,4.0,0.00,NaN,NaN,NaN,NaN,3.25,0.00,...,3.25,Cash,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,4667
2991910,2017-07-27 11:45:00,2017-07-27 11:45:00,6.0,0.00,NaN,NaN,NaN,NaN,3.25,0.00,...,8.75,Cash,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,3436
2991978,2017-07-27 11:45:00,2017-07-27 11:45:00,11.0,0.00,NaN,NaN,NaN,NaN,3.25,2.81,...,21.56,Credit Card,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,3436


In [27]:
nominute[nominute['Fare']<=3.25]

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,new_TaxiID
29,2017-04-25 15:45:00,2017-04-25 15:45:00,25.0,0.00,NaN,NaN,NaN,NaN,3.25,0.00,...,31.25,Credit Card,American United,NaN,NaN,NaN,NaN,NaN,NaN,3996
41,2017-04-24 18:00:00,2017-04-24 18:00:00,1.0,0.00,NaN,NaN,NaN,NaN,3.25,0.00,...,3.25,Cash,American United,NaN,NaN,NaN,NaN,NaN,NaN,2512
54,2017-04-27 11:30:00,2017-04-27 11:30:00,20.0,0.00,NaN,NaN,21.0,21.0,3.25,0.65,...,4.90,Credit Card,American United,41.938666,-87.711211,POINT (-87.7112105933 41.93866619620001),41.938666,-87.711211,POINT (-87.7112105933 41.93866619620001),4463
61,2017-04-27 11:00:00,2017-04-27 11:00:00,19.0,0.00,NaN,NaN,NaN,NaN,3.25,0.65,...,4.90,Credit Card,American United,NaN,NaN,NaN,NaN,NaN,NaN,4463
62,2017-04-27 11:15:00,2017-04-27 11:15:00,18.0,0.00,NaN,NaN,NaN,NaN,3.25,0.65,...,4.90,Credit Card,American United,NaN,NaN,NaN,NaN,NaN,NaN,4463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2986910,2017-07-28 20:00:00,2017-07-28 20:00:00,5.0,0.01,NaN,NaN,NaN,NaN,3.25,0.00,...,3.25,Cash,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,4667
2987061,2017-07-28 20:30:00,2017-07-28 20:30:00,4.0,0.00,NaN,NaN,NaN,NaN,3.25,0.00,...,3.25,Cash,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,4667
2991910,2017-07-27 11:45:00,2017-07-27 11:45:00,6.0,0.00,NaN,NaN,NaN,NaN,3.25,0.00,...,8.75,Cash,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,3436
2991978,2017-07-27 11:45:00,2017-07-27 11:45:00,11.0,0.00,NaN,NaN,NaN,NaN,3.25,2.81,...,21.56,Credit Card,Chicago Carriage Cab Corp,NaN,NaN,NaN,NaN,NaN,NaN,3436


## hier noch manhaten distance umrechnen in meilen fehlt noch, wenn man das macht haben wir alle daten (bis auf koordinaten komplett), sind aber nur zeilen also kann man auch löschen

In [55]:
man=df[df['Trip Miles'].isna()]

In [56]:
man

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,...,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,new_TaxiID
2058130,2017-03-08 16:00:00,2017-03-08 16:15:00,660.0,NaN,NaN,NaN,28.0,27.0,13.00,0.00,...,14.00,Cash,Chicago Medallion Leasing INC,41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.878914,-87.705897,POINT (-87.7058971305 41.8789144956),3113
3794727,2017-06-06 14:30:00,2017-06-06 14:30:00,60.0,NaN,1.703183e+10,1.703183e+10,21.0,21.0,3.25,0.00,...,8.75,Cash,Choice Taxi Association,41.942859,-87.717504,POINT (-87.717503858 41.942859303),41.942859,-87.717504,POINT (-87.717503858 41.942859303),1331
4554486,2017-07-06 23:15:00,2017-07-06 23:30:00,600.0,NaN,NaN,NaN,14.0,16.0,10.00,0.00,...,10.00,Cash,Dispatch Taxi Affiliation,41.968069,-87.721559,POINT (-87.7215590627 41.968069),41.953582,-87.723452,POINT (-87.7234523905 41.9535821253),5389
5651429,2017-09-17 03:00:00,2017-09-17 03:15:00,780.0,NaN,1.703122e+10,1.703133e+10,22.0,33.0,18.75,4.65,...,23.40,Credit Card,Dispatch Taxi Affiliation,41.921126,-87.699754,POINT (-87.6997544065 41.9211259143),41.859350,-87.617358,POINT (-87.6173580061 41.859349715),3695


In [58]:
def calculate_manhattan_distance(start_x, start_y, end_x, end_y):
    distance = abs(end_x - start_x) + abs(end_y - start_y)
    return distance

In [59]:
man['dist']=man['Trip Miles']
for i in range(4):
    man['dist'].iloc[i]=calculate_manhattan_distance(man['Pickup Centroid Latitude'].iloc[i],man['Pickup Centroid Longitude'].iloc[i],man['Dropoff Centroid Latitude'].iloc[i],man['Dropoff Centroid Longitude'].iloc[i],)

<ipython-input-59-e7a045dec163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  man['dist']=man['Trip Miles']
/Users/SG/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/SG/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [60]:
man

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,...,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,new_TaxiID,dist
2058130,2017-03-08 16:00:00,2017-03-08 16:15:00,660.0,NaN,NaN,NaN,28.0,27.0,13.00,0.00,...,Cash,Chicago Medallion Leasing INC,41.874005,-87.663518,POINT (-87.6635175498 41.874005383),41.878914,-87.705897,POINT (-87.7058971305 41.8789144956),3113,0.047289
3794727,2017-06-06 14:30:00,2017-06-06 14:30:00,60.0,NaN,1.703183e+10,1.703183e+10,21.0,21.0,3.25,0.00,...,Cash,Choice Taxi Association,41.942859,-87.717504,POINT (-87.717503858 41.942859303),41.942859,-87.717504,POINT (-87.717503858 41.942859303),1331,0.000000
4554486,2017-07-06 23:15:00,2017-07-06 23:30:00,600.0,NaN,NaN,NaN,14.0,16.0,10.00,0.00,...,Cash,Dispatch Taxi Affiliation,41.968069,-87.721559,POINT (-87.7215590627 41.968069),41.953582,-87.723452,POINT (-87.7234523905 41.9535821253),5389,0.016380
5651429,2017-09-17 03:00:00,2017-09-17 03:15:00,780.0,NaN,1.703122e+10,1.703133e+10,22.0,33.0,18.75,4.65,...,Credit Card,Dispatch Taxi Affiliation,41.921126,-87.699754,POINT (-87.6997544065 41.9211259143),41.859350,-87.617358,POINT (-87.6173580061 41.859349715),3695,0.144173


In [106]:
df.columns

Index(['Trip Start Timestamp', 'Trip End Timestamp', 'Trip Seconds',
       'Trip Miles', 'Pickup Census Tract', 'Dropoff Census Tract',
       'Pickup Community Area', 'Dropoff Community Area', 'Fare', 'Tips',
       'Tolls', 'Extras', 'Trip Total', 'Payment Type', 'Company',
       'Pickup Centroid Latitude', 'Pickup Centroid Longitude',
       'Pickup Centroid Location', 'Dropoff Centroid Latitude',
       'Dropoff Centroid Longitude', 'Dropoff Centroid  Location',
       'new_TaxiID'],
      dtype='object')

In [107]:
df_small=df[['Trip Start Timestamp', 'Trip End Timestamp', 'Trip Seconds',
       'Trip Miles', 'Fare', 'Tips',
       'Tolls', 'Extras', 'Trip Total', 'Payment Type', 'Company',
       'Pickup Centroid Latitude', 'Pickup Centroid Longitude',
        'Dropoff Centroid Latitude',
       'Dropoff Centroid Longitude',
       'new_TaxiID','dur_min']]

In [108]:
df_small

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Fare,Tips,Tolls,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Dropoff Centroid Latitude,Dropoff Centroid Longitude,new_TaxiID
0,2017-05-12 21:00:00,2017-05-12 21:15:00,865.0,4.6,15.00,2.55,0.0,2.0,19.55,Credit Card,American United,41.884987,-87.620993,41.929047,-87.651311,2464
1,2017-05-13 10:00:00,2017-05-13 10:30:00,1588.0,13.5,35.25,8.81,0.0,0.0,44.06,Credit Card,American United,NaN,NaN,NaN,NaN,3143
2,2017-05-13 13:45:00,2017-05-13 14:15:00,2184.0,12.7,35.50,0.00,0.0,5.0,40.50,Cash,American United,NaN,NaN,NaN,NaN,3143
3,2017-05-13 14:30:00,2017-05-13 14:30:00,214.0,0.4,4.75,0.00,0.0,0.0,4.75,Cash,American United,NaN,NaN,NaN,NaN,3143
4,2017-05-13 16:30:00,2017-05-13 16:45:00,1217.0,7.8,22.75,4.75,0.0,1.0,28.50,Credit Card,American United,NaN,NaN,NaN,NaN,3143
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6195692,2017-10-08 16:30:00,2017-10-08 17:45:00,4260.0,28.5,72.25,0.00,0.0,0.0,72.25,Dispute,Choice Taxi Association,41.871016,-87.631407,41.785999,-87.750934,758
6195693,2017-10-22 18:00:00,2017-10-22 18:15:00,540.0,0.0,7.25,1.65,0.0,1.0,9.90,Credit Card,Blue Ribbon Taxi Association Inc.,41.884987,-87.620993,41.885281,-87.657233,4246
6195694,2017-10-23 11:15:00,2017-10-23 11:15:00,360.0,0.0,6.00,3.00,0.0,0.0,9.00,Credit Card,Blue Ribbon Taxi Association Inc.,41.922686,-87.649489,41.944227,-87.655998,2906
6195695,2017-10-07 13:45:00,2017-10-07 14:00:00,900.0,3.1,12.25,2.65,0.0,1.0,15.90,Credit Card,Star North Management LLC,41.892508,-87.626215,41.893216,-87.637844,4986


In [109]:
df_small.count()

Trip Start Timestamp          5916168
Trip End Timestamp            5916168
Trip Seconds                  5916168
Trip Miles                    5916164
Fare                          5916168
Tips                          5916168
Tolls                         5916168
Extras                        5916168
Trip Total                    5916168
Payment Type                  5916168
Company                       5916168
Pickup Centroid Latitude      5297278
Pickup Centroid Longitude     5297278
Dropoff Centroid Latitude     5218640
Dropoff Centroid Longitude    5218640
new_TaxiID                    5916168
dtype: int64

In [102]:
test=df_small[df_small['Trip Total']>1000]
delete=test[test['Trip Seconds']<18000].index

## hier die wetter daten zum datensatz mergen (funktioniert komplett)

In [115]:
df

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,...,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Pickup Centroid Location,Dropoff Centroid Latitude,Dropoff Centroid Longitude,Dropoff Centroid Location,new_TaxiID,dur_min
0,2017-05-12 21:00:00,2017-05-12 21:15:00,865.0,4.6,1.703132e+10,1.703107e+10,32.0,7.0,15.00,2.55,...,Credit Card,American United,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.929047,-87.651311,POINT (-87.6513108767 41.9290469366),2464,14.42
1,2017-05-13 10:00:00,2017-05-13 10:30:00,1588.0,13.5,NaN,NaN,NaN,NaN,35.25,8.81,...,Credit Card,American United,NaN,NaN,NaN,NaN,NaN,NaN,3143,26.47
2,2017-05-13 13:45:00,2017-05-13 14:15:00,2184.0,12.7,NaN,NaN,NaN,NaN,35.50,0.00,...,Cash,American United,NaN,NaN,NaN,NaN,NaN,NaN,3143,36.40
3,2017-05-13 14:30:00,2017-05-13 14:30:00,214.0,0.4,NaN,NaN,NaN,NaN,4.75,0.00,...,Cash,American United,NaN,NaN,NaN,NaN,NaN,NaN,3143,3.57
4,2017-05-13 16:30:00,2017-05-13 16:45:00,1217.0,7.8,NaN,NaN,NaN,NaN,22.75,4.75,...,Credit Card,American United,NaN,NaN,NaN,NaN,NaN,NaN,3143,20.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6195692,2017-10-08 16:30:00,2017-10-08 17:45:00,4260.0,28.5,1.703184e+10,1.703198e+10,32.0,56.0,72.25,0.00,...,Dispute,Choice Taxi Association,41.871016,-87.631407,POINT (-87.6314065252 41.8710158803),41.785999,-87.750934,POINT (-87.7509342894 41.785998518),758,71.00
6195693,2017-10-22 18:00:00,2017-10-22 18:15:00,540.0,0.0,1.703132e+10,1.703183e+10,32.0,28.0,7.25,1.65,...,Credit Card,Blue Ribbon Taxi Association Inc.,41.884987,-87.620993,POINT (-87.6209929134 41.8849871918),41.885281,-87.657233,POINT (-87.6572331997 41.8852813201),4246,9.00
6195694,2017-10-23 11:15:00,2017-10-23 11:15:00,360.0,0.0,NaN,NaN,7.0,6.0,6.00,3.00,...,Credit Card,Blue Ribbon Taxi Association Inc.,41.922686,-87.649489,POINT (-87.6494887289 41.9226862843),41.944227,-87.655998,POINT (-87.6559981815 41.9442266014),2906,6.00
6195695,2017-10-07 13:45:00,2017-10-07 14:00:00,900.0,3.1,1.703108e+10,1.703108e+10,8.0,8.0,12.25,2.65,...,Credit Card,Star North Management LLC,41.892508,-87.626215,POINT (-87.6262149064 41.8925077809),41.893216,-87.637844,POINT (-87.6378442095 41.8932163595),4986,15.00


In [118]:
#wetterdatensatzladen
weather_df=pd.read_csv('weather_clean_final.csv',parse_dates=['date_time'])
weather_df=weather_df.drop('Unnamed: 0',axis=1)

In [119]:
# hier die timestamps abbrunden um gleich zusammenzufuehren
#also den datensatz ein bissche erweitern
weather_df.loc[:,'hour']=weather_df['date_time'].apply(lambda h :h.hour)
weather_df.loc[:,'day']=weather_df['date_time'].apply(lambda d :d.day)
weather_df.loc[:,'month']=weather_df['date_time'].apply(lambda m :m.month)

In [123]:
weather_df

,date_time,avg_temp,precip,hour,day,month
0,2017-01-01 00:00:00,-0.6,0.0,0,1,1
1,2017-01-01 01:00:00,-2.2,0.0,1,1,1
2,2017-01-01 02:00:00,-2.8,0.0,2,1,1
3,2017-01-01 03:00:00,-3.3,0.0,3,1,1
4,2017-01-01 04:00:00,-3.3,0.0,4,1,1
...,...,...,...,...,...,...
8755,2017-12-31 19:00:00,-11.1,0.0,19,31,12
8756,2017-12-31 20:00:00,-10.6,0.0,20,31,12
8757,2017-12-31 21:00:00,-10.6,0.0,21,31,12
8758,2017-12-31 22:00:00,-11.7,0.0,22,31,12


In [120]:
#hier soll gemerged werden, einmal werden die wetterdaten pro trip dazugeholt, dafuer hier
#erstmal neue columns erstellen mit der abgerundeten zeiten(stunde/monat/ etc)
#auch fuer spaetere analysen wichtig
df['date']=df['Trip Start Timestamp'].apply(lambda d: pd.to_datetime(d).floor('d'))
df['s_hour']=df['Trip Start Timestamp'].apply(lambda h: h.hour)
df['s_day']=df['Trip Start Timestamp'].apply(lambda d: d.day)
df['weekday']=df['Trip Start Timestamp'].apply(lambda w: w.dayofweek)
df['s_month']=df['Trip Start Timestamp'].apply(lambda m: m.month)

In [121]:
df=df.merge(weather_df[['avg_temp','precip','hour','day','month']],how='left',left_on=['s_hour','s_day','s_month'],right_on=['hour','day','month'])
df

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Pickup Census Tract,Dropoff Census Tract,Pickup Community Area,Dropoff Community Area,Fare,Tips,...,date,s_hour,s_day,weekday,s_month,avg_temp,precip,hour,day,month
0,2017-05-12 21:00:00,2017-05-12 21:15:00,865.0,4.6,1.703132e+10,1.703107e+10,32.0,7.0,15.00,2.55,...,2017-05-12,21,12,4,5,21.1,0.0,21,12,5
1,2017-05-13 10:00:00,2017-05-13 10:30:00,1588.0,13.5,NaN,NaN,NaN,NaN,35.25,8.81,...,2017-05-13,10,13,5,5,9.4,0.0,10,13,5
2,2017-05-13 13:45:00,2017-05-13 14:15:00,2184.0,12.7,NaN,NaN,NaN,NaN,35.50,0.00,...,2017-05-13,13,13,5,5,13.9,0.0,13,13,5
3,2017-05-13 14:30:00,2017-05-13 14:30:00,214.0,0.4,NaN,NaN,NaN,NaN,4.75,0.00,...,2017-05-13,14,13,5,5,17.2,0.0,14,13,5
4,2017-05-13 16:30:00,2017-05-13 16:45:00,1217.0,7.8,NaN,NaN,NaN,NaN,22.75,4.75,...,2017-05-13,16,13,5,5,21.1,0.0,16,13,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5916163,2017-10-08 16:30:00,2017-10-08 17:45:00,4260.0,28.5,1.703184e+10,1.703198e+10,32.0,56.0,72.25,0.00,...,2017-10-08,16,8,6,10,20.6,0.0,16,8,10
5916164,2017-10-22 18:00:00,2017-10-22 18:15:00,540.0,0.0,1.703132e+10,1.703183e+10,32.0,28.0,7.25,1.65,...,2017-10-22,18,22,6,10,20.0,0.0,18,22,10
5916165,2017-10-23 11:15:00,2017-10-23 11:15:00,360.0,0.0,NaN,NaN,7.0,6.0,6.00,3.00,...,2017-10-23,11,23,0,10,10.6,1.0,11,23,10
5916166,2017-10-07 13:45:00,2017-10-07 14:00:00,900.0,3.1,1.703108e+10,1.703108e+10,8.0,8.0,12.25,2.65,...,2017-10-07,13,7,5,10,20.0,0.0,13,7,10


In [122]:
#doppelte zeiten brauchen wir nicht
df.drop(['s_hour','s_day','s_month'],axis=1, inplace=True)

In [124]:
df.count()

Trip Start Timestamp          5916168
Trip End Timestamp            5916168
Trip Seconds                  5916168
Trip Miles                    5916164
Pickup Census Tract           4066643
Dropoff Census Tract          4055435
Pickup Community Area         5297080
Dropoff Community Area        5202505
Fare                          5916168
Tips                          5916168
Tolls                         5916168
Extras                        5916168
Trip Total                    5916168
Payment Type                  5916168
Company                       5916168
Pickup Centroid Latitude      5297278
Pickup Centroid Longitude     5297278
Pickup Centroid Location      5297278
Dropoff Centroid Latitude     5218640
Dropoff Centroid Longitude    5218640
Dropoff Centroid  Location    5218640
new_TaxiID                    5916168
dur_min                       5916168
date                          5916168
weekday                       5916168
avg_temp                      5916168
precip      

In [125]:
df.columns

Index(['Trip Start Timestamp', 'Trip End Timestamp', 'Trip Seconds',
       'Trip Miles', 'Pickup Census Tract', 'Dropoff Census Tract',
       'Pickup Community Area', 'Dropoff Community Area', 'Fare', 'Tips',
       'Tolls', 'Extras', 'Trip Total', 'Payment Type', 'Company',
       'Pickup Centroid Latitude', 'Pickup Centroid Longitude',
       'Pickup Centroid Location', 'Dropoff Centroid Latitude',
       'Dropoff Centroid Longitude', 'Dropoff Centroid  Location',
       'new_TaxiID', 'dur_min', 'date', 'weekday', 'avg_temp', 'precip',
       'hour', 'day', 'month'],
      dtype='object')

In [126]:
df_small=df[['Trip Start Timestamp', 'Trip End Timestamp', 'Trip Seconds',
       'Trip Miles', 'Fare', 'Tips',
       'Tolls', 'Extras', 'Trip Total', 'Payment Type', 'Company',
       'Pickup Centroid Latitude', 'Pickup Centroid Longitude',
        'Dropoff Centroid Latitude',
       'Dropoff Centroid Longitude',
       'new_TaxiID','dur_min']]

In [127]:
df_small

,Trip Start Timestamp,Trip End Timestamp,Trip Seconds,Trip Miles,Fare,Tips,Tolls,Extras,Trip Total,Payment Type,Company,Pickup Centroid Latitude,Pickup Centroid Longitude,Dropoff Centroid Latitude,Dropoff Centroid Longitude,new_TaxiID,dur_min
0,2017-05-12 21:00:00,2017-05-12 21:15:00,865.0,4.6,15.00,2.55,0.0,2.0,19.55,Credit Card,American United,41.884987,-87.620993,41.929047,-87.651311,2464,14.42
1,2017-05-13 10:00:00,2017-05-13 10:30:00,1588.0,13.5,35.25,8.81,0.0,0.0,44.06,Credit Card,American United,NaN,NaN,NaN,NaN,3143,26.47
2,2017-05-13 13:45:00,2017-05-13 14:15:00,2184.0,12.7,35.50,0.00,0.0,5.0,40.50,Cash,American United,NaN,NaN,NaN,NaN,3143,36.40
3,2017-05-13 14:30:00,2017-05-13 14:30:00,214.0,0.4,4.75,0.00,0.0,0.0,4.75,Cash,American United,NaN,NaN,NaN,NaN,3143,3.57
4,2017-05-13 16:30:00,2017-05-13 16:45:00,1217.0,7.8,22.75,4.75,0.0,1.0,28.50,Credit Card,American United,NaN,NaN,NaN,NaN,3143,20.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5916163,2017-10-08 16:30:00,2017-10-08 17:45:00,4260.0,28.5,72.25,0.00,0.0,0.0,72.25,Dispute,Choice Taxi Association,41.871016,-87.631407,41.785999,-87.750934,758,71.00
5916164,2017-10-22 18:00:00,2017-10-22 18:15:00,540.0,0.0,7.25,1.65,0.0,1.0,9.90,Credit Card,Blue Ribbon Taxi Association Inc.,41.884987,-87.620993,41.885281,-87.657233,4246,9.00
5916165,2017-10-23 11:15:00,2017-10-23 11:15:00,360.0,0.0,6.00,3.00,0.0,0.0,9.00,Credit Card,Blue Ribbon Taxi Association Inc.,41.922686,-87.649489,41.944227,-87.655998,2906,6.00
5916166,2017-10-07 13:45:00,2017-10-07 14:00:00,900.0,3.1,12.25,2.65,0.0,1.0,15.90,Credit Card,Star North Management LLC,41.892508,-87.626215,41.893216,-87.637844,4986,15.00


In [128]:
df_small.count()

Trip Start Timestamp          5916168
Trip End Timestamp            5916168
Trip Seconds                  5916168
Trip Miles                    5916164
Fare                          5916168
Tips                          5916168
Tolls                         5916168
Extras                        5916168
Trip Total                    5916168
Payment Type                  5916168
Company                       5916168
Pickup Centroid Latitude      5297278
Pickup Centroid Longitude     5297278
Dropoff Centroid Latitude     5218640
Dropoff Centroid Longitude    5218640
new_TaxiID                    5916168
dur_min                       5916168
dtype: int64

In [129]:
df.to_csv('taxi_19_may_big.csv')

In [ ]:
df_small.to_csv('taxi_19_may_small.csv')